In [1]:
import logging
import bz2
import sys
import os

In [2]:
import numpy as np

In [3]:
import qcfractal.interface as ptl
import tqdm
import sys
from openff.qcsubmit import workflow_components
from openff.qcsubmit.factories import OptimizationDatasetFactory
from openforcefield.topology import Molecule

In [4]:
# Need to turn off undefined stereochemistry warnings
logging.getLogger().setLevel(logging.ERROR)

In [5]:
def test_submit(qcs_ds, client):
    result = qcs_ds.submit(client)
    print(result)
    print("Total tasks:", sum(result.values()))

def test_submit_to_local_server(qcs_ds):
    print("Submitting to test-run to local server")
    client = ptl.FractalClient("localhost:7777", verify=False)
    test_submit(qcs_ds, client)
    
def test_submit_to_test_server(qcs_ds):
    from qcfractal import FractalSnowflakeHandler
    print("Submitting test-run to in-memory server")
    server = FractalSnowflakeHandler()
    test_submit(qcs_ds, server.client())


In [6]:
sdf_file = "./pubLigsNeutralGoodDensity.sdf"

molecules = Molecule.from_file(sdf_file,
                          "sdf",
                          allow_undefined_stereo=True,
                         )
# Considering molecules that have rotors greater than 3 for fragmentation down the line
mols = []
for molecule in molecules:
    if len(molecule.find_rotatable_bonds()) > 3:
        mols.append(molecule)
    else:
        continue
print("\nFiles loaded; molecules generated.")


Files loaded; molecules generated.


In [7]:
# Generate the workflow to apply to the molecules
qcs_ds = OptimizationDatasetFactory()

component = workflow_components.WBOFragmenter()
component.keep_non_rotor_ring_substituents = True
qcs_ds.add_workflow_component(component)

component = workflow_components.EnumerateStereoisomers()
component.max_isomers = 100
component.toolkit = "openeye"
qcs_ds.add_workflow_component(component)

component = workflow_components.StandardConformerGenerator()
component.max_conformers = 100
component.toolkit = "openeye"
component.rms_cutoff = 3.0
qcs_ds.add_workflow_component(component)

In [8]:
dataset = qcs_ds.create_dataset(
        dataset_name="Genentech PDB Ligand Expo fragment optimization neutral v1.0",
        molecules=mols,
        description="Genentech PDB Ligand Expo optimization dataset with neutral molecules, molecules with rotors > 3 are fragmented",
        tagline="Optimization set")
print("Workflow complete; dataset generated.")

WBOFragmenter                 :   0%|                   | 0/519 [00:00<?, ?it/s]

20:26:58 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:58 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:58 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:59 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:59 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:59 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:59 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:59 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:59 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:59 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:59 WARNING fragmenter: Cannot calculate WBO for fragment [H:19][C:4]1([C@@H:3]([C@H:5]([C@@:7]2([C:1](=[O:11])[NH:9][C:2](=[O:12])[NH:10]2)[O:13][C:6]1([H:21])[C:8]([H:22])([H:23])[O:17][H:29])[OH:16])[OH:14])[OH:15]. Continue growing fragment
20:26:59 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:26:59 WARN

20:27:02 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 2 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 7 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:27:02 WARNING fragmenter: A new stereocenter formed at atom 7 C
20:27:02 WARNING fragmenter: A new stereocenter formed at at

20:27:03 WARNING fragmenter: A new stereocenter formed at atom 2 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 2 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 2 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 2 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:27:03 WARNING fragmenter: A new stereocenter formed at atom 1 C


/home/maverick/anaconda3/envs/qcsub_user/lib/python3.7/site-packages/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atoms [(13, 'C')], problematic bonds [], SMILES: Cc1cn(c(=O)[nH]c1=O)[C@H]2C[C@@H]([C@@H]3C2C3)O
  problematic_bonds, oechem.OEMolToSmiles(molecule)))


...

20:44:09 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:44:09 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:44:09 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:44:09 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:44:12 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:44:12 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:44:12 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:44:12 WARNING fragmenter: A new stereocenter formed at atom 21 C


WBOFragmenter                 :  98%|████████▊| 508/519 [17:18<00:18,  1.71s/it]

20:44:18 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:44:18 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:44:18 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:44:18 WARNING fragmenter: A new stereocenter formed at atom 21 C


Preparation                   : 100%|███████| 2319/2319 [00:55<00:00, 41.83it/s]

Workflow complete; dataset generated.


In [9]:
dataset.metadata.short_description = "Optimization set"
dataset.metadata.long_description_url = "https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2020-12-02-Genentech-PDB-Ligand-Expo-fragment-optimization-neutral-v1.0"
dataset.metadata.long_description = "Genentech PDB Ligand Expo neutral molecules set"

confs = np.array([len(mol.conformers) for mol in dataset.molecules])
print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of conformers             ", dataset.n_records)
print("Number of conformers min mean max", 
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

dataset.export_dataset("dataset.json.bz2")

Number of unique molecules        2319
Number of filtered molecules      98
Number of conformers              2366
Number of conformers min mean max 1   1.02 3


In [10]:
%time test_submit_to_test_server(dataset)

Submitting test-run to in-memory server
{'default': 2363}
Total tasks: 2363
CPU times: user 49.4 s, sys: 2.46 s, total: 51.9 s
Wall time: 1min 29s


In [13]:
from openff.qcsubmit.datasets import OptimizationDataset
dataset = OptimizationDataset.parse_file('./dataset.json.bz2')
dataset.metadata.long_description = "This dataset contains Genentech PDB Ligand Expo neutral set of molecules that fragments molecules with greater than 3 rotors"
dataset.metadata.submitter = 'pavankum'
dataset.export_dataset("dataset.json.bz2")

In [15]:
from openff.qcsubmit.datasets import OptimizationDataset
ds = OptimizationDataset.parse_file('./dataset.json.bz2')
ds.visualize('visualize.pdf')